In [1]:
%pip install import-ipynb
%pip install openpyxl
%pip install xlrd

import import_ipynb
import load_data
import clean_column_names
# import write_file_as_output
# import normalize_strings
# import fill_missing_values
# import text_to_numeric
# import drop_columns
import pandas as pd
import os
import re


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Data Cleansing Functions

In [2]:
# LOAD DATA
def load_file(file_path):
    print('Filepath: ',file_path)

    # converts filepath extension to lowercase
    ext = os.path.splitext(file_path)[-1].lower()

    # Handle .csv files
    """pd.read_csv expects a file path as its argument - above line of code will not work
    because it is being passed a DataFrame/ Series object created by load file method and saved
    to variable named 'file'
    """
    if ext in ['.csv']:
        return pd.read_csv(file_path)

    # Handle excel spreadsheet files
    elif ext in ['.xls', '.xlsx']:
        # return pd.read_excel(file_path, engine="openpyxl")|pd.read_excel(file_path, engine="xlrd")
        return pd.read_excel(file_path, engine="openpyxl")
    
    # All other files
    else:
        raise ValueError("Unsupported file type")


    

In [10]:
# CREATE DATAFRAME FROM FILE AT SPECIFIED FILEPATH: NOTE- THIS CALLS load_file() function
def create_dataframe(file_path):
    # df_1 = ORIGINAL DF
    # df_2 = DataFrame to be cleansed
    
    df_1 = load_file(file_path)

    #Create FILE TO BE CLEANSED via deep copy of original df to new variable 
    df_2 = df_1.copy()
    print(f"FILE READY FOR CLEANSING\n")
    
    print(df_2.info(),"\n")
    print(df_2.shape, "\n")
    print(df_2.describe, "\n")

    return df_2


# FUNCTION CALL create_dataframe() function - NOTE: df_2 stores deep copied file on which actual data cleansing happens!
df_2 = create_dataframe(r'C:\Users\Cessn\OneDrive\Desktop\sample_datasets\data_scientist\csv_files\DataScientist2.csv')


Filepath:  C:\Users\Cessn\OneDrive\Desktop\sample_datasets\data_scientist\csv_files\DataScientist2.csv
FILE READY FOR CLEANSING

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3912 entries, 0 to 3911
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         3912 non-null   int64  
 1   index              3912 non-null   int64  
 2   Job Title          3911 non-null   object 
 3   Salary Estimate    3912 non-null   object 
 4   Job Description    3911 non-null   object 
 5   Rating             3912 non-null   float64
 6   Company Name       3912 non-null   object 
 7   Location           3912 non-null   object 
 8   Headquarters       3911 non-null   object 
 9   Size               3912 non-null   object 
 10  Founded            3912 non-null   int64  
 11  Type of ownership  3911 non-null   object 
 12  Industry           3910 non-null   object 
 13  Sector             3912 non-null   obje

In [11]:
# FILL / REMOVE BLANKS AND/OR NULLS (N/A VALUES)
# NOTE: ACTION TAKEN DEPENDS ON CONTEXT OF DF/ ROW/ COLUMN AND/OR CLIENT PREFERENCE:

'''
Get count of na (Not a Number values) - df_copy.isna().sum¶
Get count of null (null values) - df_copy.isnull().sum
Get count of null / na values fpr specific column - df["column name"].isna() / df['column name'].isnull()
For multiple columns pass a list of column names in - df.[["list_of_column_names"]].isna()
'''
def handle_blank_and_null_values(df_2):
    print(df_2.isnull(), "\n")
    print(df_2.isna(), "\n")

    # Return sum of all null values in all columns
    print(df_2.isnull().sum(), "\n")
    print(df_2.isna().sum(), "\n")

    # Note parameter inplace=True modifies current working copy of dataframe
    # Otherwise will return a new datafram w/ requested information
    # df_2.dropna(inplace=True, subset=["Job Title", "Job Description", "Industry"])
    # df_2.dropna(inplace=True) - CONSULT W/ CLIENT BEFORE DROPPING ANYTHING!

    for column in df_2:
        if df_2[column].dtype == 'object':
            df_2[column].fillna("N/A", inplace=True)
        else:
            df_2[column].fillna(0, inplace=True)

    

# CALL FUNCTION
# NOTE: Argument passed in is df_2 w/ intent to overwrite existing version of df_2 at each step of cleansing process
df_2 = handle_blank_and_null_values(df_2)
print(df_2.isnull().sum(), "\n")
print(df_2.isna().sum(), "\n")

      Unnamed: 0  index  Job Title  Salary Estimate  Job Description  Rating  \
0          False  False      False            False            False   False   
1          False  False      False            False            False   False   
2          False  False      False            False            False   False   
3          False  False      False            False             True   False   
4          False  False      False            False            False   False   
...          ...    ...        ...              ...              ...     ...   
3907       False  False      False            False            False   False   
3908       False  False      False            False            False   False   
3909       False  False      False            False            False   False   
3910       False  False      False            False            False   False   
3911       False  False      False            False            False   False   

      Company Name  Location  Headquart

C:\Users\Cessn\AppData\Local\Temp\ipykernel_5488\2935816653.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_2[column].fillna(0, inplace=True)
C:\Users\Cessn\AppData\Local\Temp\ipykernel_5488\2935816653.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when d

AttributeError: 'NoneType' object has no attribute 'isnull'

In [ ]:
# CLEAN COLUMN NAMES
''' Captures all column names in DataFrame + Does following actions:
1. Strip whitespace (both left/right side of string)
2. Convert column names to all lowercase
3. Locates regex pattern specified (replaces any char that is NOT a letter or number with an underscore)
4. str.strip('_') removes any underscore characters at start/ end of string 
(https://www.w3schools.com/python/ref_string_strip.asp)
'''
def clean_column_names(df):
    df.columns = (
        df.columns.str.strip()
                  .str.lower()
                  .str.replace(r'[^a-z0-9]+', '_', regex=True)
                  # .str.strip('_')
    )
    return df

# CALL FUNCTION:

In [3]:
# WRITE CLEANSED FILE AS OUTPUT 
def output_file(df):
    # MAKE SURE TO LIST FULL PATH TO OUTPUT FILE OTHERWISE FILL WILL BE WRITTEIN IN DIRECTORY IN WHICH THIS FILE RESIDES!
    df.to_csv(r'C:\Users\Cessn\OneDrive\Desktop\sample_datasets\city_of_baton_rouge\cats\Bus_Stop_CLEANSED.csv')
    print(f"Cleaned data saved!")

In [26]:
df_2.isnull().sum()

Unnamed: 0           0
index                0
Job Title            1
Salary Estimate      0
Job Description      1
Rating               0
Company Name         0
Location             0
Headquarters         1
Size                 0
Founded              0
Type of ownership    1
Industry             2
Sector               0
Revenue              0
Competitors          3
Easy Apply           0
dtype: int64

In [28]:
df_2['Job Title'].fillna('N/A', inplace=True)
df_2["Job Title"].isnull().sum

<bound method Series.sum of 0       False
1       False
2       False
3       False
4       False
        ...  
3907    False
3908    False
3909    False
3910    False
3911    False
Name: Job Title, Length: 3912, dtype: bool>

In [29]:
df_2[df_2["Job Title"].str.contains('scientist', case=False)]

,Unnamed: 0,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,0,0,Senior Data Scientist,$111K-$181K (Glassdoor est.),"ABOUT HOPPER\n\nAt Hopper, we’re on a mission ...",3.5,Hopper\n3.5,"New York, NY","Montreal, Canada",501 to 1000 employees,2007,Company - Private,Travel Agencies,Travel & Tourism,Unknown / Non-Applicable,-1,-1
1,1,1,"Data Scientist, Product Analytics",$111K-$181K (Glassdoor est.),"At Noom, we use scientifically proven methods ...",4.5,Noom US\n4.5,"New York, NY","New York, NY",1001 to 5000 employees,2008,Company - Private,"Health, Beauty, & Fitness",Consumer Services,Unknown / Non-Applicable,-1,-1
5,5,5,Data Scientist,$111K-$181K (Glassdoor est.),Job Brief\n\nThe ideal candidate will have pre...,2.9,IFG Companies\n2.9,"New York, NY","Hartford, CT",201 to 500 employees,1985,NaN,Insurance Carriers,Insurance,Unknown / Non-Applicable,"Colony Specialty, Markel, RLI",-1
8,8,8,AI Scientist,$111K-$181K (Glassdoor est.),Paige is a software company helping pathologis...,5.0,Paige\n5.0,"New York, NY","New York, NY",1 to 50 employees,2018,Company - Private,Enterprise Software & Network Solutions,Information Technology,Unknown / Non-Applicable,-1,TRUE
10,10,10,Data Scientist,$111K-$181K (Glassdoor est.),Company Description:\n\nQuartet is a pioneerin...,3.9,Quartet Health\n3.9,"New York, NY","New York, NY",201 to 500 employees,2014,Company - Private,Enterprise Software & Network Solutions,Information Technology,Unknown / Non-Applicable,-1,TRUE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3884,3884,4355,Senior Research Scientist - RI IPP Cooper,$55K-$112K (Glassdoor est.),JOB POSTING – Senior Research Scientist – RI I...,3.7,Nationwide Children's Hospital\n3.7,"Columbus, OH","Columbus, OH",10000+ employees,1892,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$1 to $2 billion (USD),-1,-1
3893,3893,4364,Biotransformation Scientist and DMPK Design Lead,$55K-$112K (Glassdoor est.),"Help us transform patients' lives.\n\nAt UCB, ...",3.7,UCB\n3.7,"Slough, OH","Brussel, Belgium",5001 to 10000 employees,-1,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$2 to $5 billion (USD),AbbVie,-1
3895,3895,4366,Senior/Principal Scientist - Display Technolog...,$55K-$112K (Glassdoor est.),"Help us transform patients' lives.\n\nAt UCB, ...",3.7,UCB\n3.7,"Slough, OH","Brussel, Belgium",5001 to 10000 employees,-1,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$2 to $5 billion (USD),AbbVie,-1
3908,3908,4379,Patient Safety Physician or Safety Scientist -...,$55K-$112K (Glassdoor est.),"Help us transform patients' lives.\nAt UCB, we...",3.7,UCB\n3.7,"Slough, OH","Brussel, Belgium",5001 to 10000 employees,-1,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$2 to $5 billion (USD),AbbVie,-1


In [6]:


# Copy of original data file
# file_toBe_cleansed = df.copy()

# df_2 = load_file(r'C:\Users\Cessn\OneDrive\Desktop\sample_datasets\city_of_baton_rouge\cats\Bus_Stop_COPY.xlsx')
# print("FILE READY FOR CLEANSING")
# print("FILE LOADED: \n\n ", file_toBe_cleansed, "\n")

#df_2 = clean_column_names.clean_column_names(file_toBe_cleansed)
# print("CLEANSED FILE: \n" ,df_2, "\n")

#df_2 = normalize_strings.normalize_strings(file_toBe_cleansed)
# print("CLEANSED FILE w/ strings normalized: \n" ,df_2 ,"\n")

# #df_2 = fill_missing_values.fill_missing(file_toBe_cleansed)
# print("CLEANSED FILE w/ missing values filled: \n" ,df_2 ,"\n")

# #df_2 = text_to_numeric.text_to_numeric(file_toBe_cleansed)
# print("CLEANSED FILE w/ numbers converted to numeric values: \n" ,df_2 ,"\n")

# df_2 = drop_columns.drop_columns(file_toBe_cleansed)
# print("CLEANSED FILE: COLUMNS DROPPED: \n" ,df_2 ,"\n")

# write_file_as_output.output_file(df_2)




# 8 Most Common Data Cleansing Techniques:
## 1. Dropping Columns:
cols_to_drop = ["column_names_to_be_dropped"] > df_copy.drop(to_drop, axis = 1) NOTE: axis = 0 (rows) axis = 1 (columns)

In [7]:
# # Drop irrelevant/ unnecessary columns THIS BECOMES AN EXAMPLE!
# cols_to_drop = ["ROUTE NO. 1",
#                'ROUTE NO. 2','ROUTE NO. 3',
#                'ROUTE NO. 4','ROUTE NO. 5','ROUTE NO. 6',
#                 'ROUTE NO. 7','ROUTE NO. 8','ROUTE NO. 9','ROUTE NO. 10',
#                "GEOMETRY",]

# df_2 = df_2.drop(cols_to_drop, axis = 1)
# output_file(df_2)

In [8]:
# FUNCTION
def drop_columns(df_2):
    cols_to_drop = ["ROUTE NO. 1",
               'ROUTE NO. 2','ROUTE NO. 3',
               'ROUTE NO. 4','ROUTE NO. 5','ROUTE NO. 6',
                'ROUTE NO. 7','ROUTE NO. 8','ROUTE NO. 9','ROUTE NO. 10',
               "GEOMETRY",]
    df_2 = df_2.drop(cols_to_drop, axis = 1)
    return df_2

In [9]:
# CALL DROP COLUMNS FUNCTION
df_2 = drop_columns(df_2)

# 2. Removing Duplicates
## Info - https://www.w3schools.com/python/pandas/ref_df_duplicated.asp

Syntax: <br>
get duplicates returned in DataFrame Format: df_copy[df_copy.duplicated()] <br>
get duplicates returned in Text Format: df_copy.duplicated()

# Sum total number of duplicate rows in DataFrame BEFORE dropping dupes
# Write this to report for client later (When getting ready to add reporting capabilities) <br>


## SYNTAX EXAMPLES: <br>
#  Sum total number of duplicate rows in DataFrame BEFORE Dropping dupes (Reporting purposes): df_2.duplicated().sum()

# Drop duplicates - NOTE use of inplace parameter will perform action on existing DataFrame. If inplace parameter is omitted drop_duplicates() function will return a new DataFrame object that then needs to be assigned: <br>
df_2.drop_duplicates(inplace=True) - drops duplicates on original DataFrame "inplace"

# Note use of assignment operator (=) - dropping duplicates in pandas returns a new DataFrame object cleansed of duplicate values UNLESS the inplace=True parameter is set. Notice abscence of inplace parameter in example below: <br>
df_2 = df_2.drop_duplicates()


In [10]:
def drop_duplicates(df_2):
    df_2 = df_2.drop_duplicates()

    # Write cleansed file to output
    output_file(df_2)
    
    return df_2

In [11]:
# CALL DROP DUPLICATES FUNCTION
df_2 = drop_duplicates(df_2)
df_2

Cleaned data saved!


,STOP ID,NAME,BENCH,SHELTER,SHELTER TYPE,RIDERSHIP ON,RIDERSHIP OFF,RIDERSHIP TOTAL,STOP DIRECTION,ROUTE NO.
0,1193.0,Mills @ Scenic Hwy,NB,NO,NaN,511.0,463.0,974.0,S,11.0
1,2824.0,Cedarcrest Ave @ Seracedar St,NaN,NO,NaN,NaN,NaN,NaN,S,58.0
2,1191.0,Woodpecker @ Merganzer,NaN,NO,NaN,479.0,153.0,632.0,S,11.0
3,1177.0,Blount @ Tallow,NaN,NO,NaN,319.0,460.0,779.0,W,11.0
4,1593.0,Acadian @ Oswego,NaN,NO,NaN,166.0,440.0,606.0,N,20.0
...,...,...,...,...,...,...,...,...,...,...
1694,1105.0,Scenic @ Scotlandville Library,NaN,YES,REFURBISHED,1814.0,790.0,2604.0,S,10.0
1695,1996.0,Florida @ Eugene,NaN,NO,NaN,824.0,3722.0,4546.0,W,44.0
1696,1514.0,East Airport @ Goodwood,NB,NO,NaN,123.0,117.0,240.0,W,18.0
1697,1386.0,Thomas H Delpit @ Taft,NaN,NO,NaN,163.0,41.0,204.0,S,14.0


# 3. Removing/ Dropping Irrelevant/ Unnecessary Rows
Syntax Ex: <br>
Check for Jobs located in State of Ohio in DataFrame: <br>
df_copy['Location'].str.contains("Oh", case=False) <br> <br>
df_copy['solumn_Name'].str.contains("specified_String", case=False) <br>


# 4. Removing Nulls and Blanks

## https://www.w3schools.com/python/pandas/ref_df_dropna.asp <br>
NOTE: df_dropna() method offers more flexibility in what is dropped. <br>
## https://www.w3schools.com/python/pandas/ref_df_drop.asp <br>


## Get count of na (Not a Number values) - df_copy.isna().sum <br>

## Get count of null (null values) - df_copy.isnull().sum<br>
## Get count of null / na values fpr specific column - df["column name"].isna() / df['column name'].isnull()<br>
## For multiple columns pass a list of column names in - df.[["list_of_column_names"]].isna()


In [11]:
# Example: Drop rows where all values are null/blank
df_2.dropna(axis=0, how='all', inplace=True)

In [12]:
output_file(df_2)

Cleaned data saved!


# 4B - Filling Nulls / Blanks
## https://www.w3schools.com/python/pandas/ref_df_fillna.asp

In [13]:
# Cast value as appropriate data type before assignment
# df_2.loc[0, ['RIDERSHIP ON', 'RIDERSHIP OFF', 'RIDERSHIP TOTAL']] = float(0)

for column in df_2:
    if df_2[column].dtype == 'object':
        df_2[column].fillna("N/A", inplace=True)
    df_2[column].fillna(0, inplace=True)

C:\Users\Cessn\AppData\Local\Temp\ipykernel_16712\130094186.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cleansed_file[column].fillna(0, inplace=True)
C:\Users\Cessn\AppData\Local\Temp\ipykernel_16712\130094186.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example,

In [14]:
output_file(df_2)

Cleaned data saved!


# 5 - Standardizing Values (6:26)

# SYNTAX Examples: <br>
# NOTE use df.loc as shown below to get specific column info: <br>
df_copy.loc[row_indexer, ["column_name"]] <br>
df_copy.loc[0:, ["column_name"]]

In [15]:
# NOTE use df.loc as shown below to get specific column info
# df_copy.loc[0:, ["Job Title"]]

In [16]:
# Return all elements in Job Title column where Job Title string contains a specific word
# df_copy[df_copy["Job Title"].str.contains('scientist', case=False)]


In [17]:
# Standardize all strings containing word scientist to read "Data Scientist"
# df_copy.loc[df_copy["Job Title"].str.contains("scientist", case=False), "Job Title"] = "Data Scientist"

In [18]:
# Check to ensure fields in Job Title Column have been updated
# df_copy.loc[:, "Job Title"]

In [19]:
# Capture /show all strings containing the letter k or K in column Salary Estimate 
# df_copy.loc[df_copy["Salary Estimate"].str.contains("k", case=False), "Salary Estimate"]

In [20]:
# Replace letter "k" Salary Estimate column with 000 
# Final value: 181K - 182K > 181000 - 182000

# Use dictionay object as per below syntax examples:

# replace_dict ={"\$": "", 
#                "K": "000",
#               }
# df_copy["Salary Estimate"] = df_copy['Salary Estimate'].replace(replace_dict, regex=True)

In [21]:
# df_copy["Salary Estimate"]

# 6 - Splitting Columns 8:16
## Documentation: https://pandas.pydata.org/docs/reference/api/pandas.Series.str.split.html <br>
## Youtube: https://youtu.be/FbFQH_RNMu0?t=496

## Example Code: https://docs.google.com/document/d/15Ajf-MKtOWItDITu-kukbEeshtCyMC86htXgTiMZJOA/edit?tab=t.0

## SYNTAX EXAMPLES:


# 7 - Convert Datatypes <br>
## Youtube: https://youtu.be/FbFQH_RNMu0?t=613 <br>
## Code Examples: https://docs.google.com/document/d/1HiZYEBH_5RaNjvvVtOodOFB0z-ji_4kx-kF9pv1IE-U/edit?tab=t.0 <br>

## Convert datatype: multiple columns to datatype int: <br>
convert_dtypes ={'colName_1': 'int',
                'colName_2': 'int',} <br>
df = df.astype(convert_dtypes) <br>

# CONVERT datatype of single column <br>
df['colName_1'] = df['colName_1'].astype("specify_datatype")

In [24]:
# CONVERT RIDERSHIP columns to data type INT from datatype FLOAT:
convert_dtypes ={'RIDERSHIP ON': 'int',
                'RIDERSHIP OFF': 'int',
                'RIDERSHIP TOTAL': 'int',}
df_2 = df_2.astype(convert_dtypes)

In [25]:
output_file(df_2)

Cleaned data saved!


# 8 - Stripping Strings <br>
## Youtube - https://youtu.be/FbFQH_RNMu0?t=648 <br>

## SYNTAX EXAMPLES
## Cleaning special / non-word characters from end of string: <br>
df = dfloc[:, 'column_Name'].str[:-4] - NOTE this cleans up to the last 4 characters of the string

In [9]:
test_df = pd.read_csv(r'C:\Users\Cessn\OneDrive\Desktop\sample_datasets\data_scientist\csv_files\DataScientist2.csv')
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3912 entries, 0 to 3911
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         3912 non-null   int64  
 1   index              3912 non-null   int64  
 2   Job Title          3911 non-null   object 
 3   Salary Estimate    3912 non-null   object 
 4   Job Description    3911 non-null   object 
 5   Rating             3912 non-null   float64
 6   Company Name       3912 non-null   object 
 7   Location           3912 non-null   object 
 8   Headquarters       3911 non-null   object 
 9   Size               3912 non-null   object 
 10  Founded            3912 non-null   int64  
 11  Type of ownership  3911 non-null   object 
 12  Industry           3910 non-null   object 
 13  Sector             3912 non-null   object 
 14  Revenue            3912 non-null   object 
 15  Competitors        3909 non-null   object 
 16  Easy Apply         3912 

In [16]:
for colName in test_df.columns:
    if colName == "Company Name":
        print(True)
    else:
        print(not True)

False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
